In [ ]:
import azureml.core
from azureml.core import Workspace, Datastore, Dataset

mlonazure_ws = Workspace.from_config()

mlonazure_ds = mlonazure_ws.get_default_datastore()

print('Workspace Name: ' + mlonazure_ws.name, 
      'Resource Group: ' + mlonazure_ws.resource_group,
      'Default Storage Account Name: ' + mlonazure_ds.account_name,
      'AzureML Core Version: ' + azureml.core.VERSION,
      sep = '\n')

In [ ]:
from azureml.core.compute import AmlCompute

compute_target = AmlCompute(mlonazure_ws, 'amlcluster2c-4gb')

print('Compute Name:', compute_target.name)

In [ ]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep

source_directory = 'Scripts'

pyscriptStep = PythonScriptStep(name="pystep",
                         script_name="helloworld.py", 
                         compute_target=compute_target, 
                         source_directory=source_directory,
                         allow_reuse=False)

print("Python Script Step Created")

In [ ]:
from azureml.pipeline.steps import RScriptStep

source_directory = 'Scripts'

rscriptStep = RScriptStep(name="rstep",
                         script_name="helloworld.R", 
                         compute_target=compute_target, 
                         source_directory=source_directory,
                         allow_reuse=False)

print("R Script Step Created")

In [ ]:
from azureml.pipeline.core import Pipeline

amlpipeline = Pipeline(workspace=mlonazure_ws, 
         steps=[pyscriptStep,rscriptStep],
         description= 'Hello World from Python and R!')

In [ ]:
from azureml.core import Experiment 

experiment_name = 'Pipeline-Helloworld'
experiment = Experiment(workspace = mlonazure_ws, name = experiment_name)

experiment.tag('Author', 'mlonazure')
experiment.tag('Department', 'Intro')
experiment.tag('Executor', 'Manual')

exp_pipelinerun = experiment.submit(amlpipeline)

exp_pipelinerun.wait_for_completion()

In [ ]:
publishedPipeline = amlpipeline.publish(
                                    name='HelloWorld', 
                                    description='AML Pipeline for Python and R', 
                                    version=None, 
                                    continue_on_step_failure=False)

In [ ]:
from azureml.pipeline.core import PipelineEndpoint

pipelineEndPoint = PipelineEndpoint.publish(workspace=mlonazure_ws, name="HelloWorldEndPoint",
                                            pipeline=publishedPipeline, description="Pipeline EndPoint for Python and R")
pipelineEndPoint

#Note that once you have published a pipeline, you can add other pipelines to it using, pipelineEndPoint.add(publishedPipeline)